In [77]:
import os
import pandas as pd
import json
import ast

from tqdm import tqdm

In [7]:
!python -V

Python 3.7.9


In [8]:
# Read Taxononomies
narration_verb_taxonomy_frame = pd.read_csv('narration_verb_taxonomy.csv')
narration_noun_taxonomy_frame = pd.read_csv('narration_noun_taxonomy.csv')
with open('fho_lta_taxonomy.json','r') as fho_lta_taxonomy_json:
    fho_lta_taxonomy = json.load(fho_lta_taxonomy_json)

In [9]:
nouns = []
for idx, r in narration_noun_taxonomy_frame.iterrows():
    g = ast.literal_eval(r['group'])
    for i in g: 
        nouns.append(i)
nouns = list(set(nouns))
print(len(nouns))

1610


In [10]:
narration_verb_taxonomy_frame.head()

,label,group
0,"adjust_(regulate,_increase/reduce,_change)","['adjust', 'change', 'expand', 'increase/reduc..."
1,"apply_(spread,_smear)","['apply', 'plaster', 'smear', 'spread']"
2,"arrange_(straighten,_sort,_distribute,_align)","['align', 'arrange', 'disentangle', 'distribut..."
3,"attach_(plug-in,_join,_fasten,_connect,_attach)","['affix', 'attach', 'bind', 'binds', 'connect'..."
4,blow,['blow']


### Create categories.json

In [11]:
# create categories.json
categories={}
labels = []
labels_values = []
for idx, row in narration_verb_taxonomy_frame.iterrows():
    synonyms = [] if len(row.label.split("_", 1)) == 1 else row.label.split("_")[1:]
    if row.label.split("_")[0] not in labels_values:
        labels_values.append(row.label.split("_")[0])
        labels.append({'label':row.label.split("_")[0],
                       'label_type':'verb',
                       'synonyms':synonyms,
                       'group':row.group})
    
for idx, row in narration_noun_taxonomy_frame.iterrows():
    synonyms = [] if len(row.label.split("_", 1)) == 1 else row.label.split("_")[1:]
    if row.label.split("_")[0] not in labels_values:
        labels_values.append(row.label.split("_")[0])

        labels.append({'label':row.label.split("_")[0],
                       'label_type':'noun',
                       'synonyms':synonyms,
                       'group':row.group})

categories['document_version'] = "2022-03-18"
categories['labels'] = labels

# with open('categories.json','w') as json_categories_file:
#     json.dump(categories, json_categories_file, indent=4)

## Aggregate ego4d with scod data. Filter by source

In [12]:
with open('ego4d.json','r') as egojson:
    videoinfos = json.load(egojson)

In [13]:
len(videoinfos['videos'])

9611

In [14]:
videoinfos['videos'][0]

{'video_uid': '77cc4654-4eec-44c6-af05-dbdf71f9a401',
 'duration_sec': 1247.13,
 'scenarios': ['Car - commuting, road trip', 'Talking with family members'],
 'video_metadata': {'fps': 30.0,
  'num_frames': 37414,
  'video_codec': 'libvpx-vp9',
  'audio_codec': 'aac',
  'display_resolution_width': 1920,
  'display_resolution_height': 1080,
  'sample_resolution_width': 1920,
  'sample_resolution_height': 1080,
  'mp4_duration_sec': 1247.13,
  'video_start_sec': 0.0,
  'video_duration_sec': 1247.13,
  'audio_start_sec': None,
  'audio_duration_sec': None,
  'video_start_pts': 0,
  'video_duration_pts': 19155968,
  'video_base_numerator': 1,
  'video_base_denominator': 15360,
  'audio_start_pts': None,
  'audio_duration_pts': None,
  'audio_base_numerator': None,
  'audio_base_denominator': None},
 'split_em': None,
 'split_av': None,
 'split_fho': None,
 's3_path': 's3://ego4d-minnesota/public/v1/full_scale/77cc4654-4eec-44c6-af05-dbdf71f9a401',
 'manifold_path': None,
 'origin_video_id':

Create the list of ego4d videos recorded by UNICT

In [15]:
vids_unict = []
for vid in videoinfos['videos']:
    try:
        if vid['video_source']=='unict':
            vids_unict.append(vid['video_uid'])
    except:
        pass
len(vids_unict)

1457

In [16]:
narration_frame = pd.read_json('narration.json').transpose()
narration_frame_complete = narration_frame[narration_frame.status == "complete"]
narration_frame_incomplete = narration_frame[narration_frame.status != "complete"]

In [17]:
print(f"Total number of complete narrated videos: {len(narration_frame_complete)}")
print(f"Total number of incomplete narrated videos: {len(narration_frame_incomplete)}")
print(f"Total number of narrated videos: {len(narration_frame)}")

Total number of complete narrated videos: 8550
Total number of incomplete narrated videos: 1061
Total number of narrated videos: 9611


## Read SCOD data from Ego4D

In [18]:
with open('fho_scod_train.json','r') as scod_train_json:
    fho_scod_train_frame = pd.DataFrame(json.load(scod_train_json)['clips'])
with open('fho_scod_val.json','r') as scod_val_json:
    fho_scod_val_frame = pd.DataFrame(json.load(scod_val_json)['clips'])

In [19]:
print("[FHO-SCOD] Total number of train clips: %d"%len(fho_scod_train_frame))
print("[FHO-SCOD] Total number of val clips: %d"%len(fho_scod_val_frame))

[FHO-SCOD] Total number of train clips: 19071
[FHO-SCOD] Total number of val clips: 12801


In [20]:
fho_scod_val_frame.head()

,video_uid,clip_id,clip_uid,clip_parent_start_sec,clip_parent_end_sec,clip_parent_start_frame,clip_parent_end_frame,pre_frame,pnr_frame,post_frame
0,0831d387-a359-427c-832d-b306f4720f45,804,e8a2f157-a5a0-459a-a165-fe36c75ab348,0.0,219.821029,0,6594,"{'frame_number': 64, 'clip_frame_number': 64, ...","{'frame_number': 195, 'clip_frame_number': 195...","{'frame_number': 260, 'clip_frame_number': 260..."
1,0831d387-a359-427c-832d-b306f4720f45,804,e8a2f157-a5a0-459a-a165-fe36c75ab348,0.0,219.821029,0,6594,"{'frame_number': 569, 'clip_frame_number': 569...","{'frame_number': 707, 'clip_frame_number': 707...","{'frame_number': 770, 'clip_frame_number': 770..."
2,0831d387-a359-427c-832d-b306f4720f45,804,e8a2f157-a5a0-459a-a165-fe36c75ab348,0.0,219.821029,0,6594,"{'frame_number': 623, 'clip_frame_number': 623...","{'frame_number': 751, 'clip_frame_number': 751...","{'frame_number': 795, 'clip_frame_number': 795..."
3,0831d387-a359-427c-832d-b306f4720f45,804,e8a2f157-a5a0-459a-a165-fe36c75ab348,0.0,219.821029,0,6594,"{'frame_number': 670, 'clip_frame_number': 670...","{'frame_number': 796, 'clip_frame_number': 796...","{'frame_number': 835, 'clip_frame_number': 835..."
4,0831d387-a359-427c-832d-b306f4720f45,804,e8a2f157-a5a0-459a-a165-fe36c75ab348,0.0,219.821029,0,6594,"{'frame_number': 884, 'clip_frame_number': 884...","{'frame_number': 903, 'clip_frame_number': 903...","{'frame_number': 919, 'clip_frame_number': 919..."


In [21]:
filtered_fho_scod_train_frame = fho_scod_train_frame[fho_scod_train_frame.video_uid.isin(vids_unict)]
filtered_fho_scod_val_frame = fho_scod_val_frame[fho_scod_val_frame.video_uid.isin(vids_unict)]
print("[FHO-SCOD] Total number of train clip annotations fho-unict: %d"%len(filtered_fho_scod_train_frame))
print("[FHO-SCOD] Total number of val clip annotations fho-unict: %d"%len(filtered_fho_scod_val_frame))

[FHO-SCOD] Total number of train clip annotations fho-unict: 7973
[FHO-SCOD] Total number of val clip annotations fho-unict: 4933


## Read Ego4D LTA and STA data

In [22]:
annotations_dir = ''

In [23]:
with open(os.path.join(annotations_dir,'fho_lta_train.json'),'r') as lta_train_json:
    fho_lta_train_frame = pd.DataFrame(json.load(lta_train_json)['clips'])
with open(os.path.join(annotations_dir,'fho_lta_val.json'),'r') as lta_val_json:
    fho_lta_val_frame = pd.DataFrame(json.load(lta_val_json)['clips'])

In [24]:
print("[FHO-LTA] Total number of train clips: %d"%len(fho_lta_train_frame))
print("[FHO-LTA] Total number of val clips: %d"%len(fho_lta_val_frame))

[FHO-LTA] Total number of train clips: 63956
[FHO-LTA] Total number of val clips: 33149


In [25]:
filtered_fho_lta_train_frame = fho_lta_train_frame[fho_lta_train_frame.video_uid.isin(vids_unict)]
filtered_fho_lta_val_frame = fho_lta_val_frame[fho_lta_val_frame.video_uid.isin(vids_unict)]
print("[FHO-LTA] Total number of train clip annotations fho-unict: %d"%len(filtered_fho_lta_train_frame))
print("[FHO-LTA] Total number of val clip annotations fho-unict: %d"%len(filtered_fho_lta_val_frame))

[FHO-LTA] Total number of train clip annotations fho-unict: 23295
[FHO-LTA] Total number of val clip annotations fho-unict: 10908


In [26]:
with open(os.path.join(annotations_dir,'fho_sta_train.json'),'r') as sta_train_json:
    fho_sta_train_frame = pd.DataFrame(json.load(sta_train_json)['annotations'])
with open(os.path.join(annotations_dir,'fho_sta_val.json'),'r') as sta_val_json:
    fho_sta_val_frame = pd.DataFrame(json.load(sta_val_json)['annotations'])

In [27]:
with open(os.path.join(annotations_dir,'fho_sta_train.json'),'r') as sta_train_json:
    fho_sta_nouns_frame = pd.DataFrame(json.load(sta_train_json)['noun_categories'])
with open(os.path.join(annotations_dir,'fho_sta_val.json'),'r') as sta_val_json:
    fho_sta_verbs_frame = pd.DataFrame(json.load(sta_val_json)['verb_categories'])

In [28]:
print("[FHO-STA] Total number of train clips: %d"%len(fho_sta_train_frame))
print("[FHO-STA] Total number of val clips: %d"%len(fho_sta_val_frame))

[FHO-STA] Total number of train clips: 98276
[FHO-STA] Total number of val clips: 47395


In [29]:
filtered_fho_sta_train_frame = fho_sta_train_frame[fho_sta_train_frame.video_uid.isin(vids_unict)]
filtered_fho_sta_val_frame = fho_sta_val_frame[fho_sta_val_frame.video_uid.isin(vids_unict)]
print("[FHO-STA] Total number of train clip annotations fho-unict: %d"%len(filtered_fho_sta_train_frame))
print("[FHO-STA] Total number of val clip annotations fho-unict: %d"%len(filtered_fho_sta_val_frame))

[FHO-STA] Total number of train clip annotations fho-unict: 34873
[FHO-STA] Total number of val clip annotations fho-unict: 13384


## Read Ego4D Hands data

In [31]:
with open(os.path.join(annotations_dir,'fho_hands_train.json'),'r') as hands_train_json:
    fho_hands_train_frame = pd.DataFrame(json.load(hands_train_json)['clips'])
with open(os.path.join(annotations_dir,'fho_hands_val.json'),'r') as hands_val_json:
    fho_hands_val_frame = pd.DataFrame(json.load(hands_val_json)['clips'])

In [32]:
filtered_fho_hands_train_frame = fho_hands_train_frame[fho_hands_train_frame.video_uid.isin(vids_unict)]
filtered_fho_hands_val_frame = fho_hands_val_frame[fho_hands_val_frame.video_uid.isin(vids_unict)]
print("[FHO-HANDS] Total number of train clip annotations fho-unict: %d"%len(filtered_fho_hands_train_frame))
print("[FHO-HANDS] Total number of val clip annotations fho-unict: %d"%len(filtered_fho_hands_val_frame))

[FHO-HANDS] Total number of train clip annotations fho-unict: 185
[FHO-HANDS] Total number of val clip annotations fho-unict: 139


In [33]:
filtered_fho_scod_frame = pd.concat([filtered_fho_scod_train_frame, filtered_fho_scod_val_frame], axis=0).reset_index(drop=True)
filtered_fho_lta_frame = pd.concat([filtered_fho_lta_train_frame, filtered_fho_lta_val_frame], axis=0).reset_index(drop=True)
filtered_fho_sta_frame = pd.concat([filtered_fho_sta_train_frame, filtered_fho_sta_val_frame], axis=0).reset_index(drop=True)
print("[FHO-SCOD] Total number of clip annotations fho-unict: %d"%len(filtered_fho_scod_frame))
print("[FHO-LTA] Total number of clip annotations fho-unict: %d"%len(filtered_fho_lta_frame))
print("[FHO-STA] Total number of clip annotations fho-unict: %d"%len(filtered_fho_sta_frame))

[FHO-SCOD] Total number of clip annotations fho-unict: 12906
[FHO-LTA] Total number of clip annotations fho-unict: 34203
[FHO-STA] Total number of clip annotations fho-unict: 48257


In [34]:
print("[FHO-SCOD] Number of unique indexes fho-unict: %d"%filtered_fho_scod_frame.index.nunique())
print("[FHO-LTA] Number of unique indexes fho-unict: %d"%filtered_fho_lta_frame.index.nunique())
print("[FHO-STA] Number of unique indexes fho-unict: %d"%filtered_fho_sta_frame.index.nunique())

[FHO-SCOD] Number of unique indexes fho-unict: 12906
[FHO-LTA] Number of unique indexes fho-unict: 34203
[FHO-STA] Number of unique indexes fho-unict: 48257


In [35]:
filtered_scod_video_uid_values = filtered_fho_scod_frame["video_uid"].values
filtered_lta_video_uid_values = filtered_fho_lta_frame["video_uid"].values
filtered_sta_video_uid_values = filtered_fho_sta_frame["video_uid"].values

In [37]:
video_clips_uids_dict = {}
video_uid_fps = {}
for video_uid in tqdm(filtered_sta_video_uid_values):
    clips_uids = list(set(list(filtered_fho_sta_frame[filtered_fho_sta_frame.video_uid == video_uid].clip_uid.values)))
    video_clips_uids_dict[video_uid] = clips_uids
    for v in videoinfos['videos']:
        if v['video_uid'] == video_uid:
            video_uid_fps[video_uid] = v['video_metadata']['fps']

100%|██████████| 48257/48257 [06:42<00:00, 119.89it/s]


In [38]:
clip_video_dict = {}
for k in video_clips_uids_dict.keys():
    for c in video_clips_uids_dict[k]:
        clip_video_dict[c] = k

In [36]:
with open("clip_video_uids_mapping.json", "w") as jf:
    json.dump(clip_video_dict, jf, indent=4)

In [39]:
# a function to find intersection of two lists
def intersection(lst1, lst2):
    # a new list to keep track of common elements in lst1 and lst2
    lst3 = []
    #iterate over all elements in lst1
    for value in lst1:
        #check for each element of lst1 if it's present in lst2
        if value in lst2:
            #if true, add the common element to the new list
            lst3.append(value)
    #we can shorten the four lines of code above 
    #using this code - lst3 = [value for value in lst1 if value in lst2]
    return lst3
filtered_all_uid_values = intersection(intersection(filtered_scod_video_uid_values,filtered_lta_video_uid_values),filtered_sta_video_uid_values)
print("[FHO] Total number of selected annotations: %d"%len(filtered_all_uid_values))

[FHO] Total number of selected annotations: 12878


In [40]:
print(len(filtered_fho_scod_frame))
print(len(filtered_fho_lta_frame))
print(len(filtered_fho_sta_frame))

12906
34203
48257


In [41]:
filtered_all_video_uids = intersection(narration_frame_complete.index,filtered_all_uid_values) 
print("[ALL] Total number of selected annotations: %d"%len(filtered_all_video_uids))

[ALL] Total number of selected annotations: 184


In [42]:
filtered_fho_scod_frame = filtered_fho_scod_frame[filtered_fho_scod_frame.video_uid.isin(filtered_all_video_uids)]
filtered_fho_lta_frame = filtered_fho_lta_frame[filtered_fho_lta_frame.video_uid.isin(filtered_all_video_uids)]
filtered_fho_sta_frame = filtered_fho_sta_frame[filtered_fho_sta_frame.video_uid.isin(filtered_all_video_uids)]
print(len(filtered_fho_scod_frame))
print(len(filtered_fho_lta_frame))
print(len(filtered_fho_sta_frame))

8618
15640
21136


In [43]:
# Filter narration frame
indexes_to_drop = [i for i in narration_frame_complete.index if i not in filtered_all_video_uids]
filtered_narration_frame_complete = narration_frame_complete.drop(index=indexes_to_drop).drop(columns=["status"])
# Add another columns to keep track if a narration is related to an annotated action in FHO benchmark
filtered_narration_frame_complete["has_fho_annotations"] = False

In [44]:
len(filtered_fho_scod_frame)

8618

In [45]:
filtered_fho_scod_frame.head(2)

,video_uid,clip_id,clip_uid,clip_parent_start_sec,clip_parent_end_sec,clip_parent_start_frame,clip_parent_end_frame,pre_frame,pnr_frame,post_frame
157,41416e77-3b9a-4685-bd1b-ba26b5ba19b2,790,e9be1118-a5cf-4431-b2e8-e3edcfa9f949,0.0,308.0,0,9239,"{'frame_number': 48, 'clip_frame_number': 48, ...","{'frame_number': 62, 'clip_frame_number': 62, ...","{'frame_number': 69, 'clip_frame_number': 69, ..."
158,41416e77-3b9a-4685-bd1b-ba26b5ba19b2,790,e9be1118-a5cf-4431-b2e8-e3edcfa9f949,0.0,308.0,0,9239,"{'frame_number': 72, 'clip_frame_number': 72, ...","{'frame_number': 75, 'clip_frame_number': 75, ...","{'frame_number': 102, 'clip_frame_number': 102..."


In [46]:
filtered_fho_scod_frame['pre_frame_number'] = None
filtered_fho_scod_frame['pnr_frame_number'] = None
filtered_fho_scod_frame['post_frame_number'] = None
for i in range(len(filtered_fho_scod_frame)):
    condition_pre = 'frame_number' in filtered_fho_scod_frame.iloc[i]['pre_frame'].keys()
    condition_pnr = 'frame_number' in filtered_fho_scod_frame.iloc[i]['pnr_frame'].keys()
    condition_post = 'frame_number' in filtered_fho_scod_frame.iloc[i]['post_frame'].keys()
    if condition_pre and condition_pnr and condition_post:
        pre = filtered_fho_scod_frame.iloc[i]['pre_frame']['frame_number']
        pnr = filtered_fho_scod_frame.iloc[i]['pnr_frame']['frame_number']
        post = filtered_fho_scod_frame.iloc[i]['post_frame']['frame_number']
        filtered_fho_scod_frame.iloc[i, filtered_fho_scod_frame.columns.get_loc('pre_frame_number')] = pre
        filtered_fho_scod_frame.iloc[i, filtered_fho_scod_frame.columns.get_loc('pnr_frame_number')] = pnr
        filtered_fho_scod_frame.iloc[i, filtered_fho_scod_frame.columns.get_loc('post_frame_number')] = post

In [47]:
filtered_fho_scod_frame.tail(2)

,video_uid,clip_id,clip_uid,clip_parent_start_sec,clip_parent_end_sec,clip_parent_start_frame,clip_parent_end_frame,pre_frame,pnr_frame,post_frame,pre_frame_number,pnr_frame_number,post_frame_number
12896,127392a3-036a-4af6-9db9-ca00141229db,3467,a14bf5af-6d4f-45ac-8e78-4be05c63b38c,0.0,308.0,0,9239,"{'frame_number': 8821, 'clip_frame_number': 88...","{'frame_number': 8833, 'clip_frame_number': 88...","{'frame_number': 8847, 'clip_frame_number': 88...",8821,8833,8847
12897,127392a3-036a-4af6-9db9-ca00141229db,3467,a14bf5af-6d4f-45ac-8e78-4be05c63b38c,0.0,308.0,0,9239,"{'frame_number': 8937, 'clip_frame_number': 89...","{'frame_number': 8969, 'clip_frame_number': 89...","{'frame_number': 8988, 'clip_frame_number': 89...",8937,8969,8988


In [ ]:
filtered_fho_scod_frame.to_json('filtered_fho_scod_frame.json',default_handler=str, orient = 'records')

In [48]:
# For each video uid read all narrations and check if that narration is in an annotated action interval
new_narration_data = []
for narration_video_uid, narration_entry in tqdm(filtered_narration_frame_complete.iterrows()):
    
    for narration in narration_entry.narration_pass_1['narrations']:
        fho_lta_entries_frame = filtered_fho_lta_frame[
            (filtered_fho_lta_frame.video_uid == narration_video_uid)
            &(filtered_fho_lta_frame.clip_parent_start_frame+filtered_fho_lta_frame.action_clip_start_frame <= narration['timestamp_frame'])
            &(narration['timestamp_frame'] <= filtered_fho_lta_frame.clip_parent_start_frame+filtered_fho_lta_frame.action_clip_end_frame)]
            
        # Read LTA entries
        if len(fho_lta_entries_frame) > 0:
            for lta_idx, lta_entry in fho_lta_entries_frame.iterrows():
                lta_video_uid = lta_entry.video_uid
                lta_clip_uid = lta_entry.clip_uid
                narration_timestamp_frame = narration['timestamp_frame']
                narration_text = narration['narration_text']
                start_action_frame = lta_entry.clip_parent_start_frame+lta_entry.action_clip_start_frame
                end_action_frame = lta_entry.clip_parent_start_frame+lta_entry.action_clip_end_frame
                
                new_narration_data.append({
                    'video_uid':narration_video_uid,
                    'clip_uid':lta_clip_uid,
                    'narration_text':narration_text,
                    'narration_timestamp_frame':narration['timestamp_frame'],
                    'narration_timestamp_sec':narration['timestamp_sec'],
                    'action_clip_start_frame':lta_entry.action_clip_start_frame,
                    'action_clip_end_frame':lta_entry.action_clip_end_frame,
                    'action_clip_start_sec':lta_entry.action_clip_start_sec,
                    'action_clip_end_sec':lta_entry.action_clip_end_sec,
                    'action_start_frame':start_action_frame,
                    'action_end_frame':end_action_frame
                })
                
    for narration in narration_entry.narration_pass_2['narrations']:
        fho_lta_entries_frame = filtered_fho_lta_frame[
            (filtered_fho_lta_frame.video_uid == narration_video_uid)
            &(filtered_fho_lta_frame.clip_parent_start_frame+filtered_fho_lta_frame.action_clip_start_frame <= narration['timestamp_frame'])
            &(narration['timestamp_frame'] <= filtered_fho_lta_frame.clip_parent_start_frame+filtered_fho_lta_frame.action_clip_end_frame)]
            
        # Read LTA entries
        if len(fho_lta_entries_frame) > 0:
            for lta_idx, lta_entry in fho_lta_entries_frame.iterrows():
                lta_video_uid = lta_entry.video_uid
                lta_clip_uid = lta_entry.clip_uid
                narration_timestamp_frame = narration['timestamp_frame']
                narration_text = narration['narration_text']
                start_action_frame = lta_entry.clip_parent_start_frame+lta_entry.action_clip_start_frame
                end_action_frame = lta_entry.clip_parent_start_frame+lta_entry.action_clip_end_frame
                
                new_narration_data.append({
                    'video_uid':narration_video_uid,
                    'clip_uid':lta_clip_uid,
                    'narration_text':narration_text,
                    'narration_timestamp_frame':narration['timestamp_frame'],
                    'narration_timestamp_sec':narration['timestamp_sec'],
                    'action_clip_start_frame':lta_entry.action_clip_start_frame,
                    'action_clip_end_frame':lta_entry.action_clip_end_frame,
                    'action_clip_start_sec':lta_entry.action_clip_start_sec,
                    'action_clip_end_sec':lta_entry.action_clip_end_sec,
                    'action_start_frame':start_action_frame,
                    'action_end_frame':end_action_frame
                })

184it [03:53,  1.27s/it]


In [49]:
new_narration_frame = pd.DataFrame(new_narration_data)
new_narration_frame.head(2)

,video_uid,clip_uid,narration_text,narration_timestamp_frame,narration_timestamp_sec,action_clip_start_frame,action_clip_end_frame,action_clip_start_sec,action_clip_end_sec,action_start_frame,action_end_frame
0,3e08beb0-9108-4e77-b2ae-80f91ceac474,8cf5a53b-fa9c-4fe7-b767-697133d4d5b8,#C C sits on a chair in the dining room,56,1.85265,45,285,1.5,9.5,45,285
1,3e08beb0-9108-4e77-b2ae-80f91ceac474,8cf5a53b-fa9c-4fe7-b767-697133d4d5b8,#C C picks a brush on a table,164,5.48066,45,285,1.5,9.5,45,285


In [50]:
narration_groups = []

for ni,ne in new_narration_frame.iterrows():
    group = [ne.video_uid, ne.clip_uid, ne.narration_timestamp_frame, ne.narration_text]
    if group not in narration_groups:
        narration_groups.append(group)

len(narration_groups)

39437

In [51]:
new_narration_frame_grouped = []
for g in tqdm(narration_groups):
    g_values = new_narration_frame[
        (new_narration_frame.video_uid==g[0])
        &(new_narration_frame.clip_uid==g[1])
        &(new_narration_frame.narration_timestamp_frame==g[2])
        &(new_narration_frame.narration_text==g[3])
    ]
    new_narration_frame_grouped.append({
        'video_uid':g[0],
        'clip_uid':g[1],
        'narration_text':g[3],
        'narration_timestamp_frame':g[2],
        'action_clip_start_frame':min(g_values.action_clip_start_frame.values),
        'action_clip_end_frame':max(g_values.action_clip_end_frame.values),
        'action_clip_start_sec':min(g_values.action_clip_start_sec.values),
        'action_clip_end_sec':max(g_values.action_clip_end_sec.values),
        'action_start_frame':min(g_values.action_start_frame.values),
        'action_end_frame':max(g_values.action_end_frame.values)
    })
new_narration_frame_grouped = pd.DataFrame(new_narration_frame_grouped)

100%|██████████| 39437/39437 [17:01<00:00, 38.60it/s]


In [52]:
filtered_fho_scod_frame.head(1)

,video_uid,clip_id,clip_uid,clip_parent_start_sec,clip_parent_end_sec,clip_parent_start_frame,clip_parent_end_frame,pre_frame,pnr_frame,post_frame,pre_frame_number,pnr_frame_number,post_frame_number
157,41416e77-3b9a-4685-bd1b-ba26b5ba19b2,790,e9be1118-a5cf-4431-b2e8-e3edcfa9f949,0.0,308.0,0,9239,"{'frame_number': 48, 'clip_frame_number': 48, ...","{'frame_number': 62, 'clip_frame_number': 62, ...","{'frame_number': 69, 'clip_frame_number': 69, ...",48,62,69


In [53]:
new_narration_frame_grouped.head(3)

,video_uid,clip_uid,narration_text,narration_timestamp_frame,action_clip_start_frame,action_clip_end_frame,action_clip_start_sec,action_clip_end_sec,action_start_frame,action_end_frame
0,3e08beb0-9108-4e77-b2ae-80f91ceac474,8cf5a53b-fa9c-4fe7-b767-697133d4d5b8,#C C sits on a chair in the dining room,56,45,285,1.5,9.500000,45,285
1,3e08beb0-9108-4e77-b2ae-80f91ceac474,8cf5a53b-fa9c-4fe7-b767-697133d4d5b8,#C C picks a brush on a table,164,45,362,1.5,12.066667,45,362
2,3e08beb0-9108-4e77-b2ae-80f91ceac474,8cf5a53b-fa9c-4fe7-b767-697133d4d5b8,#C C touches the drawing board with his left hand,179,45,362,1.5,12.066667,45,362


In [54]:
new_narration_frame_grouped.to_json('new_narration_frame_grouped.json')
new_narration_frame.to_json('new_narration_frame.json')

In [55]:
new_narration_frame.tail(5)

,video_uid,clip_uid,narration_text,narration_timestamp_frame,narration_timestamp_sec,action_clip_start_frame,action_clip_end_frame,action_clip_start_sec,action_clip_end_sec,action_start_frame,action_end_frame
125336,c1d8c3ca-f127-4dee-95f5-41280edde54e,831a2da3-bf97-4a79-aa47-16bcdca9e3cd,#C C puts the clothes in the laundry basket,2746,91.55516,2524,2764,84.154362,92.154362,2524,2764
125337,c1d8c3ca-f127-4dee-95f5-41280edde54e,831a2da3-bf97-4a79-aa47-16bcdca9e3cd,#C C puts the clothes in the laundry basket,2746,91.55516,2587,2827,86.254362,94.254362,2587,2827
125338,c1d8c3ca-f127-4dee-95f5-41280edde54e,831a2da3-bf97-4a79-aa47-16bcdca9e3cd,#C C puts the clothes in the laundry basket,2746,91.55516,2628,2868,87.621029,95.621029,2628,2868
125339,c1d8c3ca-f127-4dee-95f5-41280edde54e,831a2da3-bf97-4a79-aa47-16bcdca9e3cd,#C C puts the clothes in the laundry basket,2746,91.55516,2648,2888,88.287695,96.287695,2648,2888
125340,c1d8c3ca-f127-4dee-95f5-41280edde54e,831a2da3-bf97-4a79-aa47-16bcdca9e3cd,#C C puts the clothes in the laundry basket,2746,91.55516,2709,2949,90.321029,98.321029,2709,2949


# Narration analysis

In [79]:
import spacy
from spacy.symbols import nsubj, VERB
import re
nlp = spacy.load("en_core_web_sm")

def get_subject_verb_pairs(t):
    doc = nlp(t)
    subject_verb_pairs = []
    for possible_subject in doc:
        if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
            subject_verb_pairs.append({possible_subject:possible_subject.head.lemma_})
    if len(subject_verb_pairs) > 0: 
        return subject_verb_pairs
    else: return None       

    
def get_preposition_object_pairs(t):
    doc = nlp(t)
    preposition_object_pairs = []
    for possible_object in doc:
        if possible_object.dep_ == 'pobj' and possible_object.head.dep_ == 'prep':
            preposition_object_pairs.append({possible_object:possible_object.head.lemma_})
    if len(preposition_object_pairs) > 0: 
        return preposition_object_pairs
    else: return None       

    
def check_verb(token):
    """Check verb type given spacy token"""
    if token.pos_ == 'VERB':
        indirect_object = False
        direct_object = False
        for item in token.children:
            if(item.dep_ == "iobj" or item.dep_ == "pobj"):
                indirect_object = True
            if (item.dep_ == "dobj" or item.dep_ == "dative"):
                direct_object = True
        if indirect_object and direct_object:
            return 'DITRANVERB'
        elif direct_object and not indirect_object:
            return 'TRANVERB'
        elif not direct_object and not indirect_object:
            return 'INTRANVERB'
        else:
            return 'VERB'
        
def check_dobj(token):
    if token.dep_ == 'dobj':
        return str(token)
    else:
        return None

def standardize_narration(t):
    if t:
        t = t.replace("#Unsure","").replace("#unsure","").replace("#Sammary","").replace("#sammary","").replace("#Summary","")
        t = t.strip()
        if not t.startswith("#"): t="#"+t
        hashtag_span = t.find("#")
        if hashtag_span != -1:
            if t[hashtag_span+1].isspace() or t[hashtag_span+1].isdigit(): t=t[:hashtag_span+1]+t[hashtag_span+2:]
        # remove extra whitespaces
        t = re.sub(' +', ' ', t)
        t = re.sub('#\w','',t)
        t = t.strip()
        if t[0].islower(): t = t[0].upper()+t[1:]
        try:
            if t[0] == 'C' and t[1].isspace(): t = 'CameraWearer' + t[1:]
        except:
            return ""
        return t.strip()
    else: return ""

In [57]:
get_preposition_object_pairs('CameraWearer puts the clothes in the laundry basket with his hand')

[{basket: 'in'}, {hand: 'with'}]

In [ ]:
new_narration_frame_grouped['standardized_narration_text'] = ""
new_narration_frame_grouped['subject_verb_pairs'] = None
new_narration_frame_grouped['preposition_object_pairs'] = None
new_narration_frame_grouped['preposition_object_pairs'] = new_narration_frame_grouped['preposition_object_pairs'].astype(object)
count = 1
for ni, ne in tqdm(new_narration_frame_grouped.iterrows()):
    std_narr_text = standardize_narration(ne.narration_text)
    subj_verb_pairs = get_subject_verb_pairs(std_narr_text)
    prep_obj_pairs = get_preposition_object_pairs(std_narr_text)
    new_narration_frame_grouped.at[ni,"standardized_narration_text"] = std_narr_text
    try:
        new_narration_frame_grouped.at[ni,"subject_verb_pairs"] = subj_verb_pairs
    except:
        new_narration_frame_grouped.at[ni,"subject_verb_pairs"] = None
    new_narration_frame_grouped.at[ni,"preposition_object_pairs"] = prep_obj_pairs
    count += 1 

34958it [06:47, 85.64it/s] 

In [ ]:
new_narration_frame_grouped.head(2)

In [ ]:
new_narration_frame_grouped['verb'] = None
new_narration_frame_grouped['subject'] = None
for ni, ne in new_narration_frame_grouped.iterrows():
    if ne.subject_verb_pairs:
        subj_verb_dict = ne.subject_verb_pairs[0]
        # Get the subject and verb from the dictionary
        subject, verb = list(subj_verb_dict.keys())[0], list(subj_verb_dict.values())[0]
        # Assign to DataFrame using .at accessor
        new_narration_frame_grouped.at[ni, "subject"] = subject
        new_narration_frame_grouped.at[ni, "verb"] = verb

In [ ]:
new_narration_frame_grouped.head(2)

In [78]:
new_narration_frame_grouped.to_json('new_narration_frame_grouped_vn.json',default_handler=str, orient = 'records')
new_narration_frame_grouped = pd.read_json('new_narration_frame_grouped_vn.json')

In [80]:
new_narration_frame_grouped['direct_object'] = None
new_narration_frame_grouped['pos_mask'] = None
new_narration_frame_grouped['tag_mask'] = None
new_narration_frame_grouped['dep_mask'] = None
new_narration_frame_grouped['verb_type'] = 'VERB'

count = 1 

for ni, ne in tqdm(new_narration_frame_grouped.iterrows()):
    if ne.verb and ne.subject:
        prev_is_verb = False
        doc = nlp(ne.standardized_narration_text)
        verb_type = "VERB"
        pos_toks, tag_toks, dep_toks = [],[],[]
        for token in doc:
            pos_toks.append(token.pos_)
            tag_toks.append(token.tag_)
            dep_toks.append(token.dep_)
            
            if token.lemma_ == ne.verb:
                prev_is_verb = True
                verb_type = check_verb(token)
                new_narration_frame_grouped.at[ni,"verb_type"] = verb_type
            
            # Phrasal verbs
            if token.dep_ == 'prt' and prev_is_verb:
                prev_is_verb = False
                print("Phrasal verb : %s"%(str(ne.verb) + "-" + str(token)))
                new_narration_frame_grouped.at[ni,"verb"] = str(ne.verb) + "-" + str(token)
                
            # Direct object
            res_dobj = check_dobj(token)
            if res_dobj:
                new_narration_frame_grouped.at[ni,"direct_object"] = res_dobj
                
        new_narration_frame_grouped.at[ni,"pos_mask"] = str(pos_toks)
        new_narration_frame_grouped.at[ni,"tag_mask"] = str(tag_toks)
        new_narration_frame_grouped.at[ni,"dep_mask"] = str(dep_toks)
    count += 1 

99it [00:00, 92.56it/s]

Phrasal verb : raise-up


314it [00:02, 155.38it/s]

Phrasal verb : pick-up
Phrasal verb : pick-up
Phrasal verb : open-up
Phrasal verb : turn-out
Phrasal verb : pick-up
Phrasal verb : pick-up
Phrasal verb : pick-up
Phrasal verb : pick-up
Phrasal verb : pick-up
Phrasal verb : take-off
Phrasal verb : pick-up


348it [00:02, 140.46it/s]

...

39393it [04:07, 113.82it/s]

Phrasal verb : pick-up
Phrasal verb : pick-up
Phrasal verb : pick-up
Phrasal verb : pick-up


39423it [04:07, 128.08it/s]

Phrasal verb : put-down
Phrasal verb : pick-up
Phrasal verb : pick-up


39437it [04:07, 159.10it/s]


In [81]:
new_narration_frame_grouped.head(2)

,video_uid,clip_uid,narration_text,narration_timestamp_frame,action_clip_start_frame,action_clip_end_frame,action_clip_start_sec,action_clip_end_sec,action_start_frame,action_end_frame,standardized_narration_text,subject_verb_pairs,preposition_object_pairs,verb,subject,direct_object,pos_mask,tag_mask,dep_mask,verb_type
0,3e08beb0-9108-4e77-b2ae-80f91ceac474,8cf5a53b-fa9c-4fe7-b767-697133d4d5b8,#C C sits on a chair in the dining room,56,45,285,1.5,9.500000,45,285,CameraWearer sits on a chair in the dining room,[{'CameraWearer': 'sit'}],"[{'chair': 'on'}, {'room': 'in'}]",sit,CameraWearer,None,"['PROPN', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP',...","['NNP', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'DT', '...","['nsubj', 'ROOT', 'prep', 'det', 'pobj', 'prep...",INTRANVERB
1,3e08beb0-9108-4e77-b2ae-80f91ceac474,8cf5a53b-fa9c-4fe7-b767-697133d4d5b8,#C C picks a brush on a table,164,45,362,1.5,12.066667,45,362,CameraWearer picks a brush on a table,[{'CameraWearer': 'pick'}],[{'table': 'on'}],pick,CameraWearer,brush,"['PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET',...","['NNP', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'NN']","['nsubj', 'ROOT', 'det', 'dobj', 'prep', 'det'...",TRANVERB


DEPS TAGS

* nsubj: Nominal subject.
* prep: Preposition.
* pobj: Object of preposition.
* NNP: Proper noun, singular.
* VBD: Verb, past tense.
* IN: Conjunction, subordinating or preposition.

In [82]:
import sys
sys.setrecursionlimit(15000)
new_narration_frame_grouped_analysis = new_narration_frame_grouped

In [83]:
for idx, row in new_narration_frame_grouped_analysis.iterrows():
    new_narration_frame_grouped_analysis.at[idx,'subject'] = str(row.subject).strip()

In [ ]:
new_narration_frame_grouped_analysis.to_json('new_narration_frame_grouped_analysis.json', default_handler=str)

In [84]:
new_narration_frame_grouped = new_narration_frame_grouped_analysis[
    (new_narration_frame_grouped_analysis['verb_type']=='TRANVERB')
    &(new_narration_frame_grouped_analysis['subject']=='CameraWearer')
    &(new_narration_frame_grouped_analysis['direct_object'])]

verbs_set = set(new_narration_frame_grouped['verb'].values)
print(f'Total number of transverbs {len(verbs_set)}')

Total number of transverbs 458


In [ ]:
new_narration_frame_grouped.to_json('new_narration_frame_grouped_spacy_populated.json', default_handler=str)

In [85]:
len(new_narration_frame_grouped)

32182

In [86]:
verb_groups = {}
for i, r in enumerate(list(narration_verb_taxonomy_frame['label'].values)):
    if r.find('(') != -1: 
        main_v,other_v = r.split("(")[0], r.split("(")[1]
        other_v = other_v.replace("/",",").rstrip(")")
        other_v = [v.replace("_"," ").strip() for v in other_v.split(",")]
        main_v = main_v.replace("_"," ").strip()
        other_v.append(main_v)
        for ov in other_v:
            verb_groups[ov] = i
    else:
        verb_groups[r] = i

In [87]:
verb_masks_dict = {}
for v in verbs_set:
    selected_entries = new_narration_frame_grouped_analysis[new_narration_frame_grouped_analysis.verb == v]
    if v in verb_groups.keys():
        verb_masks_dict[v] = {
            'pos_masks':selected_entries.pos_mask.values,
            'tag_masks':selected_entries.tag_mask.values,
            'dep_masks':selected_entries.dep_mask.values,
            'narrations':selected_entries.standardized_narration_text.values,
            'group':verb_groups[v]
        }
    else:
        verb_masks_dict[v] = {
            'pos_masks':selected_entries.pos_mask.values,
            'tag_masks':selected_entries.tag_mask.values,
            'dep_masks':selected_entries.dep_mask.values,
            'narrations':selected_entries.standardized_narration_text.values,
            'group':-1
        }

In [88]:
verb_masks_dict['take']

{'pos_masks': array(["['PROPN', 'VERB', 'DET', 'NOUN', 'NOUN']",
        "['PROPN', 'VERB', 'DET', 'NOUN']",
        "['PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'NOUN']",
        "['PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'NOUN']",
        "['PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'NOUN']",
        "['PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'NOUN']",
        "['PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'NOUN']",
        "['PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'NOUN']",
        "['PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'NOUN']",
        "['PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'NOUN']",
        "['PROPN', 'VERB', 'DET', 'NOUN']",
        "['PROPN', 'VERB', 'DET', 'NOUN']",
        "['PROPN', 'VERB', 'DET', 'NOUN']",
        "['PROPN', 'PROPN', 'VERB', 'DET', 'NOUN']",
        "['PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN']",
        "['PROPN', 'VERB', 'NOUN', 'NOUN']",
   

In [89]:
new_narration_frame_grouped.head(2)

,video_uid,clip_uid,narration_text,narration_timestamp_frame,action_clip_start_frame,action_clip_end_frame,action_clip_start_sec,action_clip_end_sec,action_start_frame,action_end_frame,standardized_narration_text,subject_verb_pairs,preposition_object_pairs,verb,subject,direct_object,pos_mask,tag_mask,dep_mask,verb_type
1,3e08beb0-9108-4e77-b2ae-80f91ceac474,8cf5a53b-fa9c-4fe7-b767-697133d4d5b8,#C C picks a brush on a table,164,45,362,1.5,12.066667,45,362,CameraWearer picks a brush on a table,[{'CameraWearer': 'pick'}],[{'table': 'on'}],pick,CameraWearer,brush,"['PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET',...","['NNP', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'NN']","['nsubj', 'ROOT', 'det', 'dobj', 'prep', 'det'...",TRANVERB
2,3e08beb0-9108-4e77-b2ae-80f91ceac474,8cf5a53b-fa9c-4fe7-b767-697133d4d5b8,#C C touches the drawing board with his left hand,179,45,362,1.5,12.066667,45,362,CameraWearer touches the drawing board with hi...,[{'CameraWearer': 'touch'}],[{'hand': 'with'}],touch,CameraWearer,board,"['PROPN', 'VERB', 'DET', 'NOUN', 'NOUN', 'ADP'...","['NNP', 'VBZ', 'DT', 'NN', 'NN', 'IN', 'PRP$',...","['nsubj', 'ROOT', 'det', 'compound', 'dobj', '...",TRANVERB


In [90]:
import ast
import hashlib

# Save initializing EASG annotations to "annotations_all"

In [ ]:
count = 1
count_narr = 0
clips_dir = 'test_clip_location'
for scod_idx, scod_raw in tqdm(filtered_fho_scod_frame.iterrows()):
    count += 1
    narr_entries = new_narration_frame_grouped[
        (new_narration_frame_grouped.video_uid == scod_raw.video_uid)
        &(new_narration_frame_grouped.clip_uid == scod_raw.clip_uid)
        &(scod_raw.pre_frame_number >= new_narration_frame_grouped.action_start_frame)
        &(scod_raw.pre_frame_number <= new_narration_frame_grouped.action_end_frame )
        &(scod_raw.pnr_frame_number >= new_narration_frame_grouped.action_start_frame)
        &(scod_raw.pnr_frame_number <= new_narration_frame_grouped.action_end_frame )
        &(scod_raw.post_frame_number >= new_narration_frame_grouped.action_start_frame)
        &(scod_raw.post_frame_number <= new_narration_frame_grouped.action_end_frame)
    ]
    if len(narr_entries)>0:

        possible_objects_of_change = []
        for critical_frame_annotation in [scod_raw.pre_frame, scod_raw.pnr_frame, scod_raw.post_frame]:
            critical_frame_annotation = ast.literal_eval(critical_frame_annotation)
            if len(critical_frame_annotation['bbox'])> 0:

                for bbox in critical_frame_annotation['bbox']:
                    if bbox['object_type'] == 'object_of_change':
                        if bbox['structured_noun'] is not None:
                            if bbox['structured_noun'].find('(') != -1: 
                                main_n, other_n = bbox['structured_noun'].split("(")[0],bbox['structured_noun'].split("(")[1]
                                other_n = other_n.replace("/",",").rstrip(")")
                                other_n = [n.replace("_"," ").strip() for n in other_n.split(",")]
                                main_n = main_n.replace("_"," ").strip()
                                other_n.append(main_n)
                                possible_objects_of_change.extend(other_n)
                            else:
                                possible_objects_of_change.append(bbox['structured_noun'].replace("_"," ").strip())


        possible_objects_of_change = list(set(possible_objects_of_change))
        filtered_narrations = []
        if len(possible_objects_of_change) > 0:
            for o in possible_objects_of_change:
                for idx_r, n in narr_entries.iterrows():
                    if o == n.direct_object:

                        new_preposition_object_pairs = []
                        if n.preposition_object_pairs:
                            for item in n.preposition_object_pairs:
                                for k in item:
                                    new_preposition_object_pairs.append(
                                        {str(k): item[k]})

                        filtered_narrations.append({
                            'narration_text':n.standardized_narration_text,
                            'subject':n.subject,
                            'verb':n.verb,
                            'direct_object':n.direct_object,
                            'preposition_object_pairs':new_preposition_object_pairs,
                            'pos_mask':n.pos_mask,
                            'tag_mask':n.tag_mask,
                            'dep_mask':n.dep_mask,
                            'action_clip_start_frame':n.action_clip_start_frame,
                            'action_clip_end_frame':n.action_clip_end_frame,
                            'action_clip_start_sec':n.action_clip_start_sec,
                            'action_clip_end_sec':n.action_clip_end_sec
                        })

            if len(filtered_narrations)>0:
                count_narr+=len(filtered_narrations)
                entry = {
                    'video_uid':scod_raw.video_uid,
                    'clip_uid':scod_raw.clip_uid,
                    'pre_frame_number':scod_raw.pre_frame_number,
                    'pnr_frame_number':scod_raw.pnr_frame_number,
                    'post_frame_number':scod_raw.post_frame_number,
                    'pre_frame_filename':scod_raw.clip_uid+"_pre.jpg",
                    'pnr_frame_filename':scod_raw.clip_uid+"_pnr.jpg",
                    'post_frame_filename':scod_raw.clip_uid+"_post.jpg",
                    'pre_frame':ast.literal_eval(scod_raw.pre_frame),
                    'pnr_frame':ast.literal_eval(scod_raw.pnr_frame),
                    'post_frame':ast.literal_eval(scod_raw.post_frame),
                    'narrations_infos':filtered_narrations
                }
                annotation_uid = hashlib.sha256(str(entry).encode()).hexdigest()
                if not os.path.isdir(os.path.join('annotations_all',annotation_uid)): os.makedirs(os.path.join('annotations_all',annotation_uid))

                with open(os.path.join('annotations_all',annotation_uid,scod_raw.clip_uid+".json"),'w') as jsonf:
                    json.dump(entry,jsonf,indent=4)

In [91]:
len(os.listdir('annotations_all'))

6250